# 2nd Level Model Structure: Reactive Stage

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.kinetics_bounded import kinetic_block_rule
from physics.energy_bounded import energy_block_rule
from physics.VLE_bounded_MPCC_T import VLE_block_rule
from physics.MPCC_T import dew_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,300+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.cat = pe.Var(within=pe.NonNegativeReals,initialize=3000) # kg
model.Q_main = pe.Var(within=pe.Reals) # MW
model.r_total_comp = pe.Var(m.COMP_TOTAL,within=pe.Reals) # kmol/s

# Construct Individual Blocks

In [5]:
model.kinetics_block = pe.Block(rule=kinetic_block_rule)

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| kinetics_block.k_FT
| kinetics_block.r_FT_total
| kinetics_block.g0_FT
| kinetics_block.alpha
| kinetics_block.r_FT_cnum
| kinetics_block.r_FT_comp
| kinetics_block.k_WGS
| kinetics_block.Ke_WGS
| kinetics_block.r_WGS
| kinetics_block.r_WGS_comp
--------------------------------------------------



In [6]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [7]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.T_VLE
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}-\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [8]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [9]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

## MPCC: T

In [10]:
model.MPCC = pe.Block(rule=dew_block_rule)

> Importing dew Blocks......
> Adding the following local variable:
------------------------------------
| MPCC.x
| MPCC.y
| MPCC.T
| MPCC.P
| MPCC.f_V
| MPCC.f_L
| MPCC.beta
| MPCC.s_L
| MPCC.s_V
| MPCC.epi
------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC.VLE_block.n_ave
| MPCC.VLE_block.n_ave_cal
| MPCC.VLE_block.Hen
| MPCC.VLE_block.Hen0
| MPCC.VLE_block.gamma
| MPCC.VLE_block.P_sat
| MPCC.VLE_block.P_sat_Y
| MPCC.VLE_block.P_sat_dY_inf
| MPCC.VLE_block.P_sat_dY0
| MPCC.VLE_block.Hen_ref
| MPCC.VLE_block.Hen0_ref
| MPCC.VLE_block.gamma_ref
| MPCC.VLE_block.V_L
| MPCC.VLE_block.V_L_dY_inf
| MPCC.VLE_block.V_L_dY0
| MPCC.VLE_block.poynting
--------------------------------------------------



## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [11]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet) + sum(model.r_total_comp[i] for i in m.COMP_TOTAL)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [12]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [13]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


## Fixing In/Product Redundent Flow

In [14]:
model.x_.fix(0)
model.y_.fix(0)
model.L['in'].fix(0)
model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_L_.fix(0)
model.H_V_.fix(0)

## Fixing Model Parameters

In [15]:
model.cat.fix(3000)
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(1)
model.z['CO'].fix(1/(1+0.8)-0/2)
model.z['H2'].fix(0.8/(1+0.8)-0/2)
model.z['C30H62'].fix(0)
model.VLE_block.n_ave.fix(20)

In [16]:
check_DOF(pe,model)

Active Equality Constraints:	 1895
Active Inequality Constraints:	 1
Active Variables:		 2071
Fixed Variables:		 173
DOF:				 3


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [17]:
model.T.setub(300+273.15)
# model.T.fix(524)

In [18]:
# model.obj = pe.Objective(expr = model.L['out'],sense=pe.maximize)
# model.obj = pe.Objective(expr = model.cat,sense=pe.minimize)
# model.obj = pe.Objective(expr = model.T - model.MPCC.pf ,sense=pe.maximize)
model.obj = pe.Objective(expr = model.T ,sense=pe.maximize)

In [19]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

  78r-5.2629596e+02 1.63e-01 9.99e+02  -0.8 0.00e+00    -  0.00e+00 4.70e-07R  4
  79r-5.2629537e+02 1.63e-01 9.97e+02  -0.8 1.23e+02    -  8.96e-02 2.10e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80r-5.2629628e+02 1.59e-01 9.46e+02  -0.8 1.24e+02    -  1.74e-01 5.10e-02f  1
  81r-5.2630838e+02 4.98e+00 7.04e+02  -0.8 9.65e+01    -  2.89e-01 2.56e-01f  1
  82r-5.2633843e+02 5.20e+00 4.07e+03  -0.8 9.24e+01    -  5.75e-01 2.29e-01f  1
  83r-5.2638501e+02 4.45e+00 5.20e+03  -0.8 8.20e+01    -  6.22e-01 2.65e-01f  1
  84r-5.2660756e+02 6.27e+00 3.39e+03  -0.8 5.56e+01    -  9.88e-01 5.78e-01f  1
  85r-5.2798510e+02 2.00e+00 4.12e+04  -0.8 3.82e+01    -  7.44e-02 7.51e-01f  1
  86r-5.2774151e+02 8.59e-01 5.93e+04  -0.8 1.08e+01    -  9.94e-01 1.00e+00f  1
  87r-5.2774146e+02 8.16e-02 4.08e+02  -0.8 7.93e-04   4.0 1.00e+00 1.00e+00h  1
  88 -5.2772880e+02 8.16e-02 1.91e+02  -1.0 1.57e+04    -  1.60e-04 3.18e-05f  2
  89 -5.2767804e+02 8.16e-02

 172 -5.2476518e+02 1.35e+01 3.36e+13  -1.0 6.76e+04    -  6.96e-03 3.12e-04h  9
 173 -5.2475263e+02 1.36e+01 4.60e+13  -1.0 6.75e+04    -  4.95e-02 3.12e-04h  9
 174 -5.2134538e+02 4.25e+02 2.69e+13  -1.0 6.28e+04    -  9.02e-03 8.36e-02w  1
 175 -5.2119112e+02 4.23e+02 2.31e+13  -1.0 3.86e+04    -  1.90e-02 6.30e-03w  1
 176 -5.2118603e+02 4.23e+02 3.00e+15  -1.0 1.55e+04    -  1.26e-01 1.65e-04w  1
 177 -5.2473932e+02 1.36e+01 4.60e+13  -1.0 6.21e+03    -  9.02e-03 3.27e-04h  8
 178 -5.2472622e+02 1.36e+01 4.60e+13  -1.0 5.93e+04    -  8.80e-02 3.19e-04h  9
 179 -5.2467804e+02 1.36e+01 4.60e+13  -1.0 3.80e+04    -  5.61e-02 1.10e-03h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180 -5.2463135e+02 1.36e+01 4.60e+13  -1.0 3.03e+04    -  3.10e-01 1.05e-03h  7
 181 -5.2449543e+02 1.36e+01 4.61e+13  -1.0 1.80e+04    -  2.04e-01 2.97e-03h  4
 182 -5.2429278e+02 1.36e+01 4.62e+13  -1.0 1.49e+04    -  2.49e-01 4.48e-03h  3
 183 -5.2414185e+02 1.36e+01

 269 -5.1988862e+02 6.85e+00 1.30e+13  -1.0 1.35e+05    -  6.06e-03 5.37e-05h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270 -5.1922738e+02 1.95e+02 5.20e+13  -1.0 1.35e+05    -  1.25e-01 2.75e-02w  1
 271 -5.1920120e+02 1.95e+02 5.46e+13  -1.0 4.11e+04    -  1.99e-03 1.10e-03w  1
 272 -5.1920093e+02 1.95e+02 2.07e+16  -1.0 3.96e+04    -  1.50e-01 1.14e-05w  1
 273 -5.1988733e+02 6.86e+00 6.35e+13  -1.0 4.27e+03    -  1.25e-01 5.37e-05h  9
 274 -5.1988603e+02 6.86e+00 7.16e+13  -1.0 1.35e+05    -  4.06e-03 5.37e-05h 10
 275 -5.1988474e+02 6.86e+00 1.01e+14  -1.0 1.35e+05    -  1.29e-02 5.37e-05h 10
 276 -5.1988345e+02 6.86e+00 1.22e+14  -1.0 1.35e+05    -  6.72e-03 5.37e-05h 10
 277 -5.1988216e+02 6.86e+00 3.17e+14  -1.0 1.34e+05    -  4.99e-02 5.37e-05h 10
 278 -5.1988086e+02 6.86e+00 3.54e+14  -1.0 1.34e+05    -  4.58e-03 5.38e-05h 10
 279 -5.1987957e+02 6.86e+00 6.00e+14  -1.0 1.34e+05    -  2.93e-02 5.38e-05h 10
iter    objective    inf_pr 

 366 -5.1979375e+02 6.87e+00 5.15e+17  -1.0 1.19e+05    -  1.20e-02 5.44e-05h 10
 367 -5.1979244e+02 6.87e+00 5.16e+17  -1.0 1.19e+05    -  1.88e-01 5.44e-05h 10
 368 -5.1979113e+02 6.87e+00 5.16e+17  -1.0 1.19e+05    -  9.70e-03 5.44e-05h 10
 369 -5.1978983e+02 6.87e+00 5.16e+17  -1.0 1.19e+05    -  3.23e-02 5.44e-05h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370 -5.1978852e+02 6.87e+00 5.16e+17  -1.0 1.18e+05    -  1.41e-02 5.44e-05h 10
 371 -5.1978721e+02 6.87e+00 5.13e+17  -1.0 1.18e+05    -  9.04e-01 5.44e-05h 10
 372 -5.1978590e+02 6.87e+00 5.13e+17  -1.0 1.18e+05    -  9.63e-03 5.44e-05h 10
 373 -5.1978460e+02 6.87e+00 5.13e+17  -1.0 1.18e+05    -  2.40e-02 5.44e-05h 10
 374 -5.1911516e+02 1.49e+02 4.99e+17  -1.0 1.17e+05    -  8.86e-02 2.79e-02w  1
 375 -5.1909679e+02 1.49e+02 4.98e+17  -1.0 3.82e+04    -  6.67e-04 7.70e-04w  1
 376 -5.1909616e+02 1.49e+02 5.08e+17  -1.0 1.16e+04    -  2.19e-04 2.62e-05w  1
 377 -5.1978329e+02 6.87e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 460 -5.1964495e+02 6.91e+00 4.40e+17  -1.0 9.89e+04    -  4.97e-02 8.81e-05h  9
 461 -5.1964285e+02 6.91e+00 4.40e+17  -1.0 9.86e+04    -  3.40e-02 8.78e-05h  9
 462 -5.1964075e+02 6.92e+00 4.38e+17  -1.0 9.84e+04    -  1.00e+00 8.74e-05h  9
 463 -5.1963866e+02 6.92e+00 4.38e+17  -1.0 9.81e+04    -  2.02e-02 8.71e-05h  9
 464 -5.1963658e+02 6.92e+00 4.38e+17  -1.0 9.79e+04    -  9.98e-02 8.68e-05h  9
 465 -5.1910555e+02 6.70e+01 4.28e+17  -1.0 9.76e+04    -  3.01e-02 2.21e-02w  1
 466 -5.1909981e+02 6.70e+01 4.66e+17  -1.0 4.94e+04    -  5.57e-03 2.40e-04w  1
 467 -5.1909942e+02 6.70e+01 5.32e+17  -1.0 8.27e+03    -  1.03e-04 1.64e-05w  1
 468 -5.1963450e+02 6.92e+00 4.38e+17  -1.0 4.20e+03    -  3.01e-02 8.65e-05h  8
 469 -5.1963244e+02 6.92e+00 4.37e+17  -1.0 9.74e+04    -  3.36e-01 8.61e-05h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 470 -5.1963038e+02 6.92e+00

 552 -5.1940677e+02 6.99e+00 5.31e+17  -1.0 7.66e+04    -  1.06e-01 2.07e-04h  7
 553 -5.1940188e+02 6.99e+00 5.32e+17  -1.0 7.63e+04    -  3.34e-02 2.04e-04h  7
 554 -5.1939706e+02 6.99e+00 5.47e+17  -1.0 7.59e+04    -  4.30e-01 2.01e-04h  7
 555 -5.1939231e+02 6.99e+00 5.48e+17  -1.0 7.55e+04    -  2.64e-02 1.98e-04h  7
 556 -5.1909299e+02 1.79e+01 5.45e+17  -1.0 7.52e+04    -  1.05e-01 1.25e-02w  1
 557 -5.1908976e+02 1.79e+01 7.09e+17  -1.0 5.41e+04    -  2.18e-03 1.35e-04w  1
 558 -5.1908912e+02 1.79e+01 9.09e+17  -1.0 4.37e+03    -  1.03e-04 2.70e-05w  1
 559 -5.1938763e+02 6.99e+00 5.52e+17  -1.0 4.21e+03    -  1.05e-01 1.95e-04h  6
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 560 -5.1938303e+02 6.99e+00 5.53e+17  -1.0 7.49e+04    -  3.38e-02 1.93e-04h  7
 561 -5.1937849e+02 6.99e+00 5.74e+17  -1.0 7.45e+04    -  4.13e-01 1.90e-04h  7
 562 -5.1937402e+02 6.99e+00 5.75e+17  -1.0 7.42e+04    -  2.61e-02 1.87e-04h  7
 563 -5.1936961e+02 7.00e+00

 646 -5.1878594e+02 1.47e+00 1.19e+12  -1.0 1.51e+04    -  4.14e-01 1.27e-03h  1
 647 -5.1877426e+02 1.47e+00 3.34e+13  -1.0 6.61e+03    -  1.38e-02 2.90e-04h  1
 648 -5.1876609e+02 1.47e+00 2.37e+16  -1.0 6.63e+03    -  1.45e-01 2.02e-04h  1
 649r-5.1876609e+02 1.47e+00 1.00e+03   0.2 0.00e+00    -  0.00e+00 4.21e-07R  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 650r-5.1970026e+02 2.22e-01 3.45e+05   0.2 3.54e+04    -  5.04e-03 2.35e-03f  1
 651 -5.1944171e+02 2.49e-01 3.94e+03  -1.0 6.87e+03    -  1.45e-01 5.94e-03h  2
 652 -5.1930180e+02 2.56e-01 1.58e+05  -1.0 6.69e+03    -  2.11e-01 3.30e-03h  2
 653 -5.1926527e+02 2.57e-01 1.09e+07  -1.0 6.55e+03    -  2.28e-01 8.79e-04h  3
 654 -5.1926134e+02 2.57e-01 1.08e+08  -1.0 4.66e+04    -  7.38e-02 1.33e-04h  5
 655 -5.1925829e+02 2.57e-01 1.90e+08  -1.0 5.26e+04    -  9.00e-03 1.09e-04h  5
 656 -5.1925397e+02 2.57e-01 6.03e+09  -1.0 3.95e+04    -  1.91e-01 1.38e-04h  5
 657 -5.1924024e+02 2.57e-01

 741 -5.1928308e+02 7.97e+00 2.15e+05  -1.0 4.42e+02    -  8.40e-01 3.87e-03h  9
 742 -5.1927936e+02 7.95e+00 3.45e+05  -1.0 9.52e+02    -  9.63e-01 1.89e-03h 10
 743 -5.1927753e+02 7.94e+00 4.42e+05  -1.0 1.56e+03    -  7.34e-01 1.73e-03h 10
 744 -5.1814633e+02 2.00e+01 4.65e+04  -1.0 1.43e+03    -  9.06e-01 9.06e-01s 22
 745 -5.1829606e+02 1.92e+01 4.38e+04  -1.0 2.96e+03    -  5.76e-02 5.76e-02s 22
 746 -5.1849215e+02 1.80e+01 3.78e+04  -1.0 2.51e+03    -  1.35e-01 1.35e-01s 22
 747 -5.1905313e+02 2.32e+01 2.08e+04  -1.0 2.38e+03    -  4.17e-01 4.17e-01s 22
 748r-5.1905313e+02 2.32e+01 1.00e+03   1.2 0.00e+00    -  0.00e+00 0.00e+00R  1
 749r-5.1905388e+02 2.47e+00 9.00e+03   1.2 1.53e+04    -  4.06e-02 9.91e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750 -5.1904257e+02 2.42e+00 7.77e+03  -1.0 8.84e+01    -  6.81e-01 2.00e-02h  1
 751 -5.1876376e+02 1.31e+00 4.84e+03  -1.0 8.82e+01    -  4.68e-01 4.95e-01h  2
 752 -5.1848538e+02 6.87e-01

 834 -5.2848238e+02 4.85e+01 1.58e+07  -3.8 1.97e+03    -  3.06e-03 1.55e-04h  1
 835 -5.2912490e+02 4.34e+01 1.16e+08  -3.8 1.96e+03    -  7.18e-03 1.31e-01f  1
 836 -5.2917801e+02 4.30e+01 1.18e+08  -3.8 1.07e+03    -  1.17e-04 1.04e-02h  1
 837 -5.2975921e+02 3.85e+01 7.98e+07  -3.8 1.38e+03    -  3.96e-02 1.14e-01h  1
 838 -5.2975933e+02 3.85e+01 7.95e+07  -3.8 1.51e+03    -  2.74e-03 2.29e-05h  1
 839 -5.3094814e+02 3.18e+01 1.88e+08  -3.8 1.50e+03    -  6.10e-04 2.42e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 840 -5.3094842e+02 3.18e+01 1.95e+08  -3.8 1.44e+03    -  7.34e-03 5.24e-05h  1
 841 -5.3163154e+02 2.82e+01 1.08e+08  -3.8 1.44e+03    -  8.70e-04 1.38e-01f  1
 842 -5.3163165e+02 2.82e+01 1.08e+08  -3.8 1.43e+03    -  4.57e-03 2.24e-05h  1
 843 -5.3389344e+02 2.39e+01 8.75e+08  -3.8 1.42e+03    -  5.15e-03 4.76e-01f  1
 844 -5.3389527e+02 2.39e+01 8.75e+08  -3.8 1.36e+03    -  2.70e-02 3.28e-04h  1
 845 -5.3435885e+02 2.21e+01

In [20]:
model.T.value

538.6452954582529

In [21]:
model.MPCC.T.value

473.1503769827638

In [22]:
model.VLE_block.T_VLE.value

473.15

In [23]:
model.L['out'].value

3.677467769700983e-13

In [24]:
sum(model.MPCC.y[i].value for i in m.COMP_TOTAL)

1.0

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}'.format(model.x[i].value,model.y[i].value))

# Iterative Solve for Data Analysis

In [ ]:
results_anchor = results
update_dual(pe,model)
model.solutions.store_to(results_anchor)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
catrange = [3000,1000,500,300,100,50,25,10]
Trange = np.linspace(300+273.15,200+273.15,26)

In [ ]:
rf_data_master = {}

for c in catrange:
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = model.cat ,sense=pe.minimize)

    model.solutions.load_from(results_anchor)
    update_dual(pe,model) 
       
    model.T.fix(300+273.15)
    model.cat.fixed = False
    model.cat.setlb(c)
    
    results_anchor = opt.solve(model,tee=False)
    model.solutions.store_to(results_anchor)
    update_dual(pe,model)
    
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = model.T ,sense=pe.maximize)
    model.T.fixed = False
    model.cat.fixed = True
    
    
    print('\nWorking on Cat = {} kg'.format(model.cat.value))
    print('-'*108)
    
    rf_data = {}
    rf_data['x'] = {};rf_data['y'] = {};
    rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; rf_data['P_VLE'] = []; 
    rf_data['s_L'] = [];
    for i in m.COMP_TOTAL:
        rf_data['x'][i] = []
        rf_data['y'][i] = []       

    for t in Trange:
        model.T.setub(t)
        results = opt.solve(model,tee=False)
        update_dual(pe,model)
        model.T.setub(300+273.15)
        print('Solved\t|T = {:.2f} K\t|V = {:.4f} kmol/s\t|L = {:.4f} kmol/s\t|s_L = {:.4f}\t|P_VLE = {:.1f}'.\
              format(model.T.value,model.V['out'].value,model.L['out'].value,model.MPCC.s_L.value,model.VLE_block.P_VLE.value))
    
        rf_data['T'].append(model.T.value)
        rf_data['Q'].append(model.Q_main.value)
        rf_data['V'].append(model.V['out'].value)
        rf_data['L'].append(model.L['out'].value)
        rf_data['P_VLE'].append(model.VLE_block.P_VLE.value)
        rf_data['s_L'].append(model.MPCC.s_L.value)

        for i in m.COMP_TOTAL:
            rf_data['x'][i].append(model.x[i].value)
            rf_data['y'][i].append(model.y[i].value)
        
    rf_data_master[c] = rf_data

# Optimization results across disappearing phases

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(16,9))
for c in catrange:
    ax1.plot(Trange,rf_data_master[c]['V'])
ax1.set_ylabel('Vapor Flow Rate (kmol/s)',fontsize=14)
ax1.set_xlabel('Temperature (K)',fontsize=14)
ax1.tick_params(labelsize=14)
ax1.grid()

for c in catrange:
    ax2.plot(Trange,rf_data_master[c]['L'])
ax2.set_ylabel('Liquid Flow Rate (kmol/s)',fontsize=14)
ax2.set_xlabel('Temperature (K)',fontsize=14)
ax2.tick_params(labelsize=14)
ax2.grid()

ax2.legend(catrange,fontsize=14)
fig.suptitle('Vapor and Liquid Flow Rate',fontsize=18)

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
for c in catrange:
    ax1.plot(Trange,np.divide(rf_data_master[c]['V'],np.add(rf_data_master[c]['V'],rf_data_master[c]['L'])))
ax1.set_ylabel('Vapor Fraction',fontsize=14)
ax1.set_xlabel('Temperature (K)',fontsize=14)
ax1.tick_params(labelsize=14)
ax1.legend(catrange,fontsize=14)
ax1.set_title('Vapor Fraction',fontsize=18)
ax1.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
for c in catrange:
    ax1.plot(Trange,rf_data_master[c]['P_VLE'])
ax1.set_ylabel(r'$P_{VLE}$ value',fontsize=14)
ax1.set_xlabel('Temperature (K)',fontsize=14)
ax1.tick_params(labelsize=14)
ax1.legend(catrange,fontsize=14)
ax1.set_title(r'$P_{VLE}$ (Bar)',fontsize=18)
ax1.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
for c in catrange:
    ax1.plot(Trange,np.multiply(rf_data_master[c]['s_L'],rf_data_master[c]['L']))
ax1.set_ylabel(r'$s_L \cdot L$ value',fontsize=14)
ax1.set_xlabel('Temperature (K)',fontsize=14)
ax1.tick_params(labelsize=14)
ax1.legend(catrange,fontsize=14)
# ax1.set_ylim(2.49e-5,2.51e-5)
ax1.set_title(r'$s_L \cdot L$',fontsize=18)
ax1.grid()
plt.show()